In [1]:
import sys
sys.path.append('/Users/yuanqingwang/Documents/GitHub/mudslide/')

In [2]:
import openmm as mm
import numpy as onp
import jax
import jax.numpy as jnp

In [5]:
from openff.toolkit.topology import Molecule
from openff.toolkit.typing.engines.smirnoff import ForceField
from openmmtools.integrators import VelocityVerletIntegrator

ff = ForceField("openff-2.0.0.offxml")
molecule = Molecule.from_smiles("Cn1cnc2c1c(=O)n(C)c(=O)n2C")
# molecule = Molecule.from_smiles("CCO")
molecule.generate_conformers(n_conformers=1)

mm_system = ff.create_openmm_system(molecule.to_topology())

from openmmtools.integrators import DummyIntegrator
from openff.units import unit as off_unit
X = (molecule.conformers[0] / off_unit.nanometers).magnitude

integrator = VelocityVerletIntegrator(0.01)
simulation = mm.app.Simulation(molecule.to_topology().to_openmm(), mm_system, integrator)
simulation.context.setPositions(X)
simulation.context.setVelocitiesToTemperature(300.0)
V = simulation.context.getVelocities(asNumpy=True).value_in_unit(off_unit.nanometers/off_unit.picoseconds) 

AttributeError: 'Context' object has no attribute 'getVelocities'

In [4]:
import mudslide as md
from mudslide.system import System
md_system = System.from_openmm(mm_system)
from mudslide.integrators import VelocityVerletIntegrator as MDVelocityVerletIntegrator
integrator = VelocityVerletIntegrator(timestep=0.1)
integrate = jax.jit(lambda state: integrator(100, state, md_system, key=jax.random.PRNGKey(0)))

/Users/yuanqingwang/Documents/GitHub/mudslide/mudslide/forces/harmonic_bond_force.py:21: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in array is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  length = jnp.array([], dtype=jnp.float64)
/Users/yuanqingwang/Documents/GitHub/mudslide/mudslide/forces/harmonic_bond_force.py:24: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in array is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  k = jnp.array([], dtype=jnp.float64)
/Users/yuanqingwang/Documents/GitHub/mudslide/mudslide/forces/harmonic_angle_force.py:26: UserW

In [35]:
from openmmtools.integrators import VelocityVerletIntegrator
integrator = VelocityVerletIntegrator(0.1 * mm.unit.picosecond)
context = mm.Context(openmm_system, integrator)
context.setPositions(X * mm.unit.nanometer)
context.setVelocities(V * mm.unit.nanometer/mm.unit.picosecond)

In [36]:
%%timeit
integrator.step(100)

2.15 ms ± 76.3 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [32]:
from mudslide.integrators.velocity_verlet import VelocityVerletIntegrator
from mudslide.system import System
from mudslide.state import State
from mudslide.forces.harmonic_bond_force import HarmonicBondForce
force = HarmonicBondForce(
    particle1=jnp.array([0]), 
    particle2=jnp.array([1]), 
    length=jnp.array([0.0]), 
    k=jnp.array([1.0]),
)
system = System(masses=jnp.array([1.0, 1.0]), forces=[force])
integrator = VelocityVerletIntegrator(timestep=0.1)
integrate = jax.jit(lambda state: integrator(100, state, system, key=jax.random.PRNGKey(0)))

In [33]:
%%timeit
state = State(jnp.array(X), jnp.array(V))
state = integrate(state)
jax.block_until_ready(state)

61.1 μs ± 6.04 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
